与学习相关的技巧

参数的更新

- 神经网络的学习的目的是找到使损失函数的值尽可能小的参数。这是寻
找最优参数的问题，解决这个问题的过程称为最优化（optimization）

- SGD（stochastic gradient descent），随机梯度下降法

In [4]:
class SGD:
    def __init__(self, lr=0.01):
            self.lr = lr
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

- SGD的缺点：如果函数的形状非均向（anisotropic），比如呈延伸状，搜索
的路径就会非常低效

- SGD低效的根本原因是，梯度的方向并没有指向最小值的方向

Momentum

In [5]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)

        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

- 实例变量v会保存物体的速度。初始化时，v中什么都不保存，但当第一次调用update()时，v会以字典型变量的形式保存与参数结构相同的数据

AdaGrad

- 学习率衰减（learning rate decay）:随着学习的进行，使学习率逐渐减

- AdaGrad会为参数的每个元素适当地调整学习率，与此同时进行学习
（AdaGrad的Ada来自英文单词Adaptive，即“适当的”的意思）

- AdaGrad会记录过去所有梯度的平方和。因此，学习越深入，更新的幅度就越小。实际上，如果无止境地学习，更新量就会变为0，完全不再更新。为了改善这个问题，可以使用RMSProp方法。RMSProp方法并不是将过去所有的梯度一视同仁地相加，而是逐渐地遗忘过去的梯度，在做加法运算时将新梯度的信息更多地反映出来。这种操作从专业上讲，称为“指数移动平均”，呈指数函数式地减小过去的梯度的尺度

In [7]:
import numpy as np

class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None

    def update(self, params, grads):
        if self.h is None:  # 初始化 h（历史梯度平方和）
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.keys():
            self.h[key] += grads[key] * grads[key]  # 累积梯度平方
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)  # 自适应学习率更新

Adam

- Adam融合了Momentum和AdaGrad的方法。通过组合前面两个方法的优点，有望
实现参数空间的高效搜索。此外，进行超参数的“偏置校正”也是Adam的特征。

- Adam会设置3个超参数。一个是学习率（论文中以α出现），另外两个是一次momentum系数β1和二次momentum系数β2。根据论文，标准的设定值是β1为0.9，β2为0.999。设置了这些值后，大多数情况下都能顺利运行

梯度消失(gradient vanishing)

- 随着层次的加深，各层的激活值呈偏向0和1的分布。这里使用的sigmoid函数是S型函数，随着输出不断地靠近0（或者靠近1），它的导数的值逐渐接近0。因此，偏向0和1的数据分布会造成反向传播中梯度的值不断变小，最后消失

- 各层的激活值的分布都要求有适当的广度。为什么呢？因为通过在各层间传递多样性的数据，神经网络可以进行高效的学习。反过来，如果传递的是有所偏向的数据，就会出现梯度消失或者“表现力受限”的问题，导致学习可能无法顺利进行

Batch Normalization

- 优点：
• 可以使学习快速进行（可以增大学习率）
• 不那么依赖初始值（对于初始值不用那么神经质）
• 抑制过拟合（降低Dropout等的必要性）

- 以进行学习时的mini-batch为单位，按mini-batch进行正规化

过拟合

- 原因：1.模型拥有大量参数、表现力强 2.模型拥有大量参数、表现力强

权值衰减

- 权值衰减是一直以来经常被使用的一种抑制过拟合的方法。该方法通过在学习的过程中对大的权重进行惩罚，来抑制过拟合。很多过拟合原本就是因为权重参数取值过大才发生的

- L2范数：各个元素的平方和

Dropout

- 在学习的过程中随机删除神经元

In [8]:
class Dropout:
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask

        else:
            return x * (1.0 - self.dropout_ratio)
        
    def backward(self, dout):
        return dout * self.mask

- 机器学习中经常使用集成学习。所谓集成学习，就是让多个模型单独进行学习，推理时再取多个模型的输出的平均值。用神经网络的语境来说，比如，准备5个结构相同（或者类似）的网络，分别进行学习，测试时，以这5个网络的输出的平均值作为答案。实验告诉我们，通过进行集成学习，神经网络的识别精度可以提高好几个百分点。这个集成学习与Dropout有密切的关系。这是因为可以将Dropout理解为，通过在学习过程中随机删除神经元，从而每一次都让不同的模型进行学习。并且，推理时，通过对神经元的输出乘以删除比例（比如，0.5等），可以取得模型的平均值。也就是说，可以理解成，Dropout将集成学习的效果（模拟地）通过一个网络实现了

超参数(hyper-parameter)

- 用于调整超参数的数据，一般称为验证数据（validation data）。我们使用这个验证数据来评估超参数的好坏


- 训练数据用于参数（权重和偏置）的学习，验证数据用于超参数的性能评估。为了确认泛化能力，要在最后使用（比较理想的是只用一次）测试数据。

- 有报告显示，在进行神经网络的超参数的最优化时，与网格搜索等有规律的搜索相比，随机采样的搜索方式效果更好。这是因为在多个超参数中，各个超参数对最终的识别精度的影响程度不同